
# Extract raw commits (`raw/ORG/REPO.commits.json`)


In [ ]:
from src.repoextractor import RepoExtractor
import json
with open("../data/processed/mirantis/repos.json", "r") as file:
    data = json.load(file)
    
    for repo in data:
        print(repo["name"])
        RepoExtractor().extract_raw_commits_from_repo(repo["name"])
    
    


# Extract commit messages and dates (`processed/ORG/commits-messages-dates/REPO.pickle`)


In [ ]:
import os
from repoextractor import RepoExtractor

for subdir, dirs, files in os.walk("../data/raw/mirantis"):
    for file in files:
        repo = file.split(".")[0]
        path = os.path.join(subdir, file)
        RepoExtractor().save_commits_messages_dates(path,
                                                    "mirantis",
                                                    repo)
        



# Extract repos with at least 2 commits per month (`processed/ORG/months-valid.json`)


In [ ]:
from src.repovalidator import RepoValidator
import os
import json

repos_bool = []
for subdir, dirs, files in os.walk("../data/processed/mirantis/commits-messages-dates"):
    for file in files:
        filename = file.split(".")[0]
        complete_path = os.path.join(subdir, file)
        repos_bool.append({"name": filename, "is_valid": RepoValidator(complete_path).has_at_least_2_commits_per_month()})
        
with open("../data/processed/mirantis/months-valid.json", "w") as f:
    valids = list(map(lambda y: y["name"], filter(lambda x: x["is_valid"], repos_bool)))
    json.dump(valids, f)


# Extract commits with their files (`/processed/ORG/commits-files/REPO.commits.pickle`)


In [ ]:
import json
from repoextractor import RepoExtractor

with open("../data/processed/mirantis/months-valid.json", "r") as file:
    extractor = RepoExtractor()
    data = json.load(file)
    for repo in data: 
        r = extractor.get_repo(f"mirantis/{repo}")
        extractor.extract_repo(r, "mirantis")
        
        



# Extract repos with at least 11% of IaC files (`processed/ORG/valid-repos.json`)


In [ ]:
from src.repovalidator import RepoValidator
import os

valids = []
for subdir, dirs, files in os.walk("../data/processed/mirantis/commits-files"):
    for file in files:
        repo = file.split(".")[0]
        path = os.path.join(subdir, file)
        if RepoValidator(path).has_11_percent_of_iac():
            valids.append(repo)
valids            

In [ ]:
import json

with open('../data/processed/mirantis/valid-repos.json', "w") as f:
    json.dump(valids, f)